# <div align="center" style="color: #ff5733;">7 11 Data Analysis</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """with mb as 
(select customer_id, right(new_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details`
union all 
select customer_id, right(old_mobile_number, 10) mobile from `risk_credit_mis.customer_contact_details` where old_mobile_number is not null
),
mb1 
as
(select distinct customer_id, mobile from mb),
mb2
as
(select *, row_number() over(partition by customer_id order by customer_id) custrank from mb1),
delqb as
(Select lmt.customerId, case when a1.obs_min_inst_def30 = 3 then lmt.customerId end obsfstpd30 
, case when a1.min_inst_def30 in (1,2,3) then lmt.customerId end fstpd30dev
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data a1
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = a1.loanAccountNumber
),delqb1 
as
(select customerId, 
max(case when obsfstpd30 is not null and fstpd30dev is not null then 1 else 0 end) fstpd30def,
max(case when obsfstpd30 is not null then 1 else 0 end) obsfpd30
from delqb where obsfstpd30 is not null
group by 1
),
base as
(select * from 
`prj-prod-dataplatform.manual_source_extracts.partner_711_payment_transcation_backtest_20240125` a1
left join mb2 on cast(mb2.mobile as numeric) = cast(a1.mobile_num as numeric)
left join delqb1 on delqb1.customerId = cast(mb2.customer_id as numeric)
)
select * from base"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1004b073-0d60-48ef-adc9-2f398ad70f4e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [3]:
print(f"The shape of the dataset is:\t {df.shape}")

The shape of the dataset is:	 (1196007, 13)


In [4]:
df.columns

Index(['mobile_num', 'txn_id', 'amount', 'sector', 'merchant_name',
       'txn_datetime', 'store_id', 'customer_id', 'mobile', 'custrank',
       'customerId', 'fstpd30def', 'obsfpd30'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196007 entries, 0 to 1196006
Data columns (total 13 columns):
 #   Column         Non-Null Count    Dtype              
---  ------         --------------    -----              
 0   mobile_num     1196007 non-null  Int64              
 1   txn_id         1196007 non-null  Int64              
 2   amount         1196007 non-null  float64            
 3   sector         1196007 non-null  object             
 4   merchant_name  1196007 non-null  object             
 5   txn_datetime   1196007 non-null  datetime64[us, UTC]
 6   store_id       1196007 non-null  Int64              
 7   customer_id    1157200 non-null  Int64              
 8   mobile         1157200 non-null  object             
 9   custrank       1157200 non-null  Int64              
 10  customerId     14062 non-null    Int64              
 11  fstpd30def     14062 non-null    Int64              
 12  obsfpd30       14062 non-null    Int64              
dtypes: Int64(8),

In [6]:
df['mobile_num'] = df['mobile_num'].astype(str)

In [7]:
df['sector'].value_counts()

sector
EMI                       831125
Loans                     149071
Bank                       76768
Prepaid Load               65593
Postpaid Land Lines        12963
Transpo or Toll            11468
Government                 10260
Utility-Water               8992
Internet                    7733
Remittance                  3897
Utility-Power               3036
Credit Card                 2908
Cable TV                    2823
Postpaid Mobile             2634
E-Commerce                  1864
E-Wallet                    1238
Airlines                     983
Gaming                       916
Collections                  498
Retailer Load                474
Insurance                    353
Tuition                      304
Other Prepaid Services       106
Name: count, dtype: int64

In [8]:
# Count matching lp_id with mobile
matching_count = df[df['mobile_num'].isin(df['mobile'])].shape[0]
# Count lp_id that are missing in mobile
missing_count = df[~df['mobile_num'].isin(df['mobile'])].shape[0]

# Total number of lp_id records
total_lp_id = df.shape[0]

# Calculate share of lp_id in mobile
share_in_mobile = (matching_count / total_lp_id) * 100

print(f"Number of lp_id matching with mobile: {matching_count}")
print(f"Number of lp_id missing in mobile: {missing_count}")
print(f"Share of lp_id in mobile: {share_in_mobile:.2f}%")


Number of lp_id matching with mobile: 1157200
Number of lp_id missing in mobile: 38807
Share of lp_id in mobile: 96.76%


In [9]:
df[df['mobile_num'].isin(df['mobile'])].to_csv("Matchingmobilenumberwithoutdatabase.csv", index = False)

In [10]:
df[~df['mobile_num'].isin(df['mobile'])].to_csv("Missingmobilenumberfromourdatabasewhenmatchingwith711data.csv", index = False)

In [11]:
df[~df['mobile_num'].isin(df['mobile'])]

,mobile_num,txn_id,amount,sector,merchant_name,txn_datetime,store_id,customer_id,mobile,custrank,customerId,fstpd30def,obsfpd30
12,9066910733,741088575,170.0,Government,NBI,2022-02-10 02:12:46+00:00,1594,<NA>,None,<NA>,<NA>,<NA>,<NA>
13,9066910733,621632785,170.0,Government,NBI,2021-08-04 18:51:50+00:00,1869,<NA>,None,<NA>,<NA>,<NA>,<NA>
14,9062812704,609031332,170.0,Government,NBI,2021-07-12 18:26:38+00:00,2682,<NA>,None,<NA>,<NA>,<NA>,<NA>
127,9061418598,831583166,8120.0,Loans,TALA,2022-07-08 17:57:06+00:00,206,<NA>,None,<NA>,<NA>,<NA>,<NA>
128,9061418598,831583897,9020.0,Loans,TALA,2022-07-08 17:58:18+00:00,206,<NA>,None,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195607,9277742582,592888031,10000.0,Loans,DRAGON LOANS,2021-06-13 03:27:48+00:00,3323,<NA>,None,<NA>,<NA>,<NA>,<NA>
1195608,9277742582,592886779,10000.0,Loans,DRAGON LOANS,2021-06-12 23:56:59+00:00,3323,<NA>,None,<NA>,<NA>,<NA>,<NA>
1195609,9277742582,592888559,2692.0,Loans,DRAGON LOANS,2021-06-12 23:58:07+00:00,3323,<NA>,None,<NA>,<NA>,<NA>,<NA>
1195799,9483115809,600479344,115.0,Prepaid Load,SMART REGULAR 115,2021-06-26 19:14:17+00:00,2917,<NA>,None,<NA>,<NA>,<NA>,<NA>


In [12]:
# Get unique lp_id and mobile values
unique_lp_ids = df['mobile_num'].unique()
unique_mobiles = df['mobile'].unique()

# Count how many unique lp_id values are in unique mobiles
matching_count = sum(lp_id in unique_mobiles for lp_id in unique_lp_ids)

# Total number of unique lp_id records
total_unique_lp_id = len(unique_lp_ids)

# Calculate share of unique lp_id in unique mobiles
share_in_mobile = (matching_count / total_unique_lp_id) * 100

print(f"Number of unique lp_id matching with unique mobile: {matching_count}")
print(f"Total number of unique lp_id: {total_unique_lp_id}")
print(f"Share of unique lp_id in unique mobile: {share_in_mobile:.2f}%")

Number of unique lp_id matching with unique mobile: 58522
Total number of unique lp_id: 60686
Share of unique lp_id in unique mobile: 96.43%


In [13]:
df.columns

Index(['mobile_num', 'txn_id', 'amount', 'sector', 'merchant_name',
       'txn_datetime', 'store_id', 'customer_id', 'mobile', 'custrank',
       'customerId', 'fstpd30def', 'obsfpd30'],
      dtype='object')

In [14]:
# Calculate sector-wise default rate
sector_default_rate = df.groupby('sector').apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())

# Calculate count of unique customers per sector
unique_customers_count = df.groupby('sector')['customer_id'].nunique()
unique_711mobile_count = df.groupby('sector')['mobile_num'].nunique()
unique_obsfstpd30_customer_count = df.groupby('sector')['customerId'].nunique()

# Combine results into a DataFrame
result_df = pd.DataFrame({
    'Sector': sector_default_rate.index,
    'Default Rate (%)': sector_default_rate.values * 100,
    'Unique Customers Count': unique_customers_count,
    'unique 711mobile count': unique_711mobile_count,
    'Observed fstpd30 customer count': unique_obsfstpd30_customer_count,
})

# Sort by Default Rate in descending order
result_df_sorted = result_df.sort_values(by='Default Rate (%)', ascending=False)

print("Sector-wise Default Rate and Unique Customers Count (sorted from highest to lowest default rate):")
sector_result = result_df_sorted.copy()
sector_result

C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_14964\2263231737.py:2: RuntimeWarning: invalid value encountered in scalar divide
  sector_default_rate = df.groupby('sector').apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())
C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_14964\2263231737.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sector_default_rate = df.groupby('sector').apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())


Sector-wise Default Rate and Unique Customers Count (sorted from highest to lowest default rate):


,Sector,Default Rate (%),Unique Customers Count,unique 711mobile count,Observed fstpd30 customer count
sector,,,,,
Government,Government,35.294118,6310,6530,92
Insurance,Insurance,33.333333,67,71,2
Utility-Power,Utility-Power,25.000000,889,923,6
Prepaid Load,Prepaid Load,14.710042,15559,16077,181
Airlines,Airlines,12.500000,625,656,14
EMI,EMI,11.227098,57736,59864,856
Cable TV,Cable TV,10.000000,771,798,10
Utility-Water,Utility-Water,6.716418,2683,2790,38
Internet,Internet,6.666667,2147,2218,27


In [15]:
# Calculate sector-wise and merchant-wise default rates
sector_merchant_default_rate = df.groupby(['sector', 'merchant_name']).apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())

# Calculate count of unique customers per sector and merchant
unique_customers_count = df.groupby(['sector', 'merchant_name'])['customer_id'].nunique()
unique_711mobile_count = df.groupby(['sector', 'merchant_name'])['mobile_num'].nunique()
unique_obsfstpd30_customer_count = df.groupby(['sector', 'merchant_name'])['customerId'].nunique()

# Combine results into a DataFrame
result_df = pd.DataFrame({
    'Sector': sector_merchant_default_rate.index.get_level_values(0),
    'Merchant': sector_merchant_default_rate.index.get_level_values(1),
    'Default Rate (%)': sector_merchant_default_rate.values * 100,
    'Unique Customers Count': unique_customers_count,
    'unique 711mobile count': unique_711mobile_count,
    'Observed fstpd30 customer count': unique_obsfstpd30_customer_count,
})

# Sort by Default Rate in descending order
result_df_sorted = result_df.sort_values(by='Default Rate (%)', ascending=False)

pd.set_option('display.max_rows', None)
print("Sector-wise and Merchant-wise Default Rate and Unique Customers Count (sorted from highest to lowest default rate):")
sector_merchant_result = result_df_sorted.copy()
sector_merchant_result

C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_14964\229032578.py:2: RuntimeWarning: invalid value encountered in scalar divide
  sector_merchant_default_rate = df.groupby(['sector', 'merchant_name']).apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())
C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_14964\229032578.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sector_merchant_default_rate = df.groupby(['sector', 'merchant_name']).apply(lambda x: x['fstpd30def'].sum() / x['obsfpd30'].sum())


Sector-wise and Merchant-wise Default Rate and Unique Customers Count (sorted from highest to lowest default rate):


Sector  \
sector                 merchant_name                                                      
Prepaid Load           UNLICALL 15                                         Prepaid Load   
                       TNT Giga Video 99                                   Prepaid Load   
Utility-Power          MECO                                               Utility-Power   
Prepaid Load           SUNXPRESSLOAD 300                                   Prepaid Load   
Utility-Power          BATELEC1                                           Utility-Power   
Prepaid Load           TM EZ70                                             Prepaid Load   
Loans                  MALAYAN BANK                                               Loans   
Utility-Water          PRIMEWATER                                         Utility-Water   
Government             DFA                                                   Government   
                       BUREAU OF QUARANTINE                                  Government   
Prepaid Load           TM ALL IN SURF 20 FB                                Prepaid Load   
                       SMART REGULAR 30                                    Prepaid Load   
                       SMART REGULAR 60                                    Prepaid Load   
Postpaid Land Lines    GLOBELINES                                   Postpaid Land Lines   
Insurance              AXAPHIL                                                Insurance   
Airlines               PHILIPPINEAIRLINES                                      Airlines   
Cable TV               SKYCABLE                                                Cable TV   
Prepaid Load           GLOBE GO+99                                         Prepaid Load   
                       GLOBE 100                                           Prepaid Load   
                       SUNXPRESSLOAD 100                                   Prepaid Load   
Government             NSOHELPLINE.COM                                       Government   
Prepaid Load           GLOBE Autoload                                      Prepaid Load   
                       TM EASYSURF 50                                      Prepaid Load   
                       GIGA SURF 99                                        Prepaid Load   
Loans                  ACOM                                                       Loans   
EMI                    HELLO MONEY                                                  EMI   
Government             NBI                                                   Government   
                       MYEG PH                                               Government   
Prepaid Load           TM SURF4ALL99TM                                     Prepaid Load   
EMI                    GCash                                                        EMI   
Airlines               CEBUPACIFIC                                             Airlines   
Prepaid Load           GLOBE GOSURF 50                                     Prepaid Load   
                       GIGA SURF 50                                        Prepaid Load   
                       TM ALL IN SURF 30                                   Prepaid Load   
Internet               CONVERGEICT                                             Internet   
Loans                  TALA                                                       Loans   
E-Commerce             SHOPEE                                                E-Commerce   
Prepaid Load           GLOBE GOSAKTO 90                                    Prepaid Load   
Remittance             Palawan Padala                                        Remittance   
Postpaid Land Lines    PLDT                                         Postpaid Land Lines   
EMI                    SHOPEEPAY (DRAGONPAY)                                        EMI   
Loans                  HOME CREDIT                                                Loans   
Bank                   UNIONBANK                                             

In [16]:
# Create a Pandas Excel writer using ExcelWriter
excel_file = 'sector_analysis.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    # Write each DataFrame to a separate sheet
    sector_result.to_excel(writer, sheet_name='Sector Result', index=False)
    sector_merchant_result.to_excel(writer, sheet_name='Sector Merchant Result', index=False)

print(f"Excel file '{excel_file}' has been created with two sheets.")

Excel file 'sector_analysis.xlsx' has been created with two sheets.
